In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import time
from sklearn.metrics.pairwise import cosine_similarity
import torch
import re

In [ ]:
df_tweet_lang2 = pd.read_csv("Latest_Tweet_Fake_Spreader_Tweets.csv", lineterminator='\n', usecols = ['Tweet_ID','User_ID', 'Tweet_Text','Tweet_Lang','Tweet_Time' ], dtype=str)

In [ ]:
df_tweet_lang2.Tweet_Time = pd.to_datetime(df_tweet_lang2.Tweet_Time, format='%Y-%m-%d %H:%M:%S.%f')
df_tweet_lang2['Date']= df_tweet_lang2.Tweet_Time.dt.date
df_tweet_lang2['Date']= pd.to_datetime(df_tweet_lang2.Date, format='%Y-%m-%d')

In [ ]:
df_tweet_lang2 = df_tweet_lang2.drop_duplicates(subset=['Tweet_ID'])
df_tweet_lang2 = df_tweet_lang2.reset_index(drop = True)

In [ ]:
len(df_tweet_lang2)

In [ ]:
df_tweet_lang2['Text_without_Link'] = df_tweet_lang2['Tweet_Text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df_tweet_lang2['Text_without_Link']=df_tweet_lang2['Text_without_Link'].str.replace('(@\w+.*?)',"")

In [ ]:
df_tweet_lang2['Text_without_Link'] = df_tweet_lang2['Text_without_Link'].replace('', np.nan)
df_tweet_lang2 = df_tweet_lang2.dropna(subset=['Text_without_Link'])
df_tweet_lang2 = df_tweet_lang2[(df_tweet_lang2.Tweet_Lang == 'en') | (df_tweet_lang2.Tweet_Lang == 'hi')]
df_tweet_lang2 = df_tweet_lang2.dropna(subset=['Text_without_Link'])
df_tweet_lang2['Text_Size'] = df_tweet_lang2.Text_without_Link.apply(lambda x: len(x))
df_tweet_lang2 = df_tweet_lang2.reset_index(drop = True)

In [ ]:
print(len(df_tweet_lang2))

In [ ]:
df_tweet_lang2.to_csv('Tweet_Info_latest_Proprocessed.csv')

In [ ]:
uid = df_tweet_lang2.groupby('User_ID').count().reset_index().User_ID.tolist()

In [ ]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
model.max_seq_length = 280
model.do_lower_case =  'True'

In [ ]:
for i in uid:
    
    df_portion = df_tweet_lang2.loc[df_tweet_lang2.User_ID == i].reset_index()
    tid = df_portion.Tweet_ID.tolist()

    try:
        embeddings = model.encode(df_portion.Text_without_Link.tolist())
        emb_df = pd.DataFrame(embeddings)
        emb_df['Tweet_ID'] = tid
        emb_df.to_csv('User_Embedding/' + str(i) + '.csv')
        print("Processed User: " + str(i))
    except:
        print("Error for user: " + str(i))
        continue
        


In [ ]:
from os import listdir
from os.path import isfile, join
mypath_input = '/home/mnemo/users/randomuser/SBERT All Embedding/User_Embedding/'
mypath_output = '/home/mnemo/users/randomuser/SBERT All Embedding/User_Spam/'
onlyfiles = [f for f in listdir(mypath_input) if isfile(join(mypath_input, f))]

In [ ]:
datafile = pd.read_csv(mypath_input + onlyfiles[0])
datafile = datafile.drop(datafile.columns[0], axis=1)

In [ ]:
datafile = datafile.drop(datafile.columns[-1], axis=1)

In [ ]:
datafile